In [ ]:
!pip install pandas

In [2]:
import re

import pandas as pd

In [3]:
def sort_columns(df):
    # Function to extract number from column name
    def extract_number(col_name):
        num = re.findall(r'\d+', col_name)
        return int(num[0]) if num else None

    # Custom sorting function
    def custom_sort(col_name):
        if col_name == 'record_ID':
            return -1
        elif col_name == 'token_encryption_key':
            return float('inf')
        else:
            num = extract_number(col_name)
            if num is not None and col_name.startswith('token_'):
                return num
            else:
                # Use a high value to place other columns at the end
                return float('inf')

    # Sort the column names using the custom sorting function
    sorted_columns = sorted(df.columns, key=custom_sort)

    # Reassign the sorted columns to the DataFrame
    return df[sorted_columns]

In [4]:
tokens = [
      ("/Users/andrewprokhorenkov/Downloads/MIDRC_N3C_UCHICAGO_20220728/MIDRC_N3C_UCHICAGO_20220728_TOKENS.csv", "|")
    , ("/Users/andrewprokhorenkov/Downloads/MIDRC_N3C_UCHICAGO_20221202/MIDRC_N3C_UCHICAGO_20221202_TOKENS.csv", ",")
    , ("/Users/andrewprokhorenkov/Downloads/MIDRC_N3C_UCHICAGO_20230105/MIDRC_N3C_UCHICAGO_20230105_TOKENS.csv", "|")
    , ("/Users/andrewprokhorenkov/Downloads/MIDRC_N3C_UCHICAGO_20230112/MIDRC_N3C_UCHICAGO_20230112_TOKENS.csv", "|")
]

In [7]:
dfs = []
for file, sep in tokens:
    df = pd.read_csv(file, sep=sep)
    dfs.append(df)
df = pd.concat(dfs)

df["ID"] = df["record_ID"].combine_first(df["ACR CASE ID"]).combine_first(df["RECORD_ID"]).combine_first(df["record_id"])
df.drop(["record_ID", "ACR CASE ID", "RECORD_ID", "record_id"], axis=1, inplace=True)
df.insert(0, "record_ID", df["ID"])
df.drop(["ID"], axis=1, inplace=True)

df = sort_columns(df)

df = df.drop_duplicates()

df.to_csv("MIDRC_N3C_UCHICAGO_20230405_TOKENS.csv", sep="|", index=False)